In [165]:
import torch
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch import nn

<h3>Loading first 1000 rows into data for basic model

In [119]:
positive_data = pd.read_csv('positive_sample.txt', header = None, nrows = 500)
positive_data.columns = ["Gene"]
negative_data = pd.read_csv('negative_sample.txt', header = None, nrows = 500)
negative_data.columns = ["Gene"]
train_data = positive_data.append(negative_data)
#print(data.head(2))

<h3>Creating word indexes for per permutations of proteins

In [154]:
vocab_set = set()

def generate_vocabmap_helper(set, k): 
    n = len(set)  
    generate_k_sized(set, "", n, k) 
def generate_k_sized(set, prefix, n, k): 
    if (k == 0) : 
        vocab_set.add(prefix)
        return
    for i in range(n): 
        newPrefix = prefix + set[i] 
        generate_k_sized(set, newPrefix, n, k - 1) 
def generate_vocabmap(n):
    alphabet = ['0','1','2','3','4']
    generate_vocabmap_helper(alphabet, n)
    
    vocab_set_1 = sorted(vocab_set)
    vocab_map = {}
    
    for i in range(len(vocab_set_1)):
        vocab_map[vocab_set_1[i]] = i
    return vocab_map
def get_labels():
    #labels = []
    #positive_
    labels = torch.cat((torch.ones([500, 1], dtype=torch.long), torch.zeros([500, 1], dtype=torch.long)))
    #print(labels)
    return labels

window_size = 1     
vocabulary = generate_vocabmap(window_size)

#print(len(vocabulary))

In [155]:
# Generate embeddings for word indexes
embeds = nn.Embedding(len(vocabulary), 5)
embeddings = {}
for word in vocabulary:
    embeddings[word] = embeds(torch.tensor(vocabulary[word], dtype=torch.long))

#print((protein_embed))

In [156]:
list_of_tensors = []
for data in train_data.itertuples():
    gene = data.Gene
    for i in range(len(gene) - window_size + 1):
        if i == 0:
            first_tensor = embeddings[gene[i:i+window_size]]
        else:
            first_tensor = torch.cat((first_tensor, embeddings[gene[i:i+window_size]]), 0)
    list_of_tensors.append(first_tensor)
inputs = torch.stack(list_of_tensors)
labels = get_labels()

In [186]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(505, 500)
        self.fc2 = nn.Linear(500, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
net = Net()

<h3> Implementing the FC Layer

In [187]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.0)

In [188]:
for epoch in range(1000):  # loop over the dataset multiple times
    running_loss = 0.0
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    #loss = criterion(outputs, labels)
#     print(outputs)
    outputs = autograd.Variable(outputs.data, requires_grad=True)

    loss = criterion(outputs, torch.max(labels,1)[1])
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    #print(running_loss)
    if epoch % 25 == 0:    #print every 25 mini-batches
        print('[%d, %5d] loss: %.9f' %
              (epoch + 1, epoch + 1, running_loss/len(inputs)))
        #running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.000485342
[26,    26] loss: 0.000485342
[51,    51] loss: 0.000485342
[76,    76] loss: 0.000485342
[101,   101] loss: 0.000485342
[126,   126] loss: 0.000485342
[151,   151] loss: 0.000485342
[176,   176] loss: 0.000485342
[201,   201] loss: 0.000485342
[226,   226] loss: 0.000485342
[251,   251] loss: 0.000485342
[276,   276] loss: 0.000485342
[301,   301] loss: 0.000485342
[326,   326] loss: 0.000485342
[351,   351] loss: 0.000485342
[376,   376] loss: 0.000485342
[401,   401] loss: 0.000485342
[426,   426] loss: 0.000485342
[451,   451] loss: 0.000485342
[476,   476] loss: 0.000485342
[501,   501] loss: 0.000485342
[526,   526] loss: 0.000485342
[551,   551] loss: 0.000485342
[576,   576] loss: 0.000485342
[601,   601] loss: 0.000485342
[626,   626] loss: 0.000485342
[651,   651] loss: 0.000485342
[676,   676] loss: 0.000485342
[701,   701] loss: 0.000485342
[726,   726] loss: 0.000485342
[751,   751] loss: 0.000485342
[776,   776] loss: 0.000485342
[801,   801] 